In [1]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
write_folder = os.path.join('/home/mplitt','2P_scratch')
s2p_write_folder = os.path.join('/home/mplitt','2P_scratch')

mouse = '4467975.5'

file_list = ({'date': '28_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '29_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':5},
             {'date': '01_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':7},
             {'date': '02_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':5},
             {'date': '04_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '04_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':8},
             {'date': '05_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3},
             {'date': '06_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':5},
             {'date': '06_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':9},
             )

In [3]:
def check_and_make_h5s(file_list, runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        try:
            source_folder  = os.path.join('/media','mplitt','Backup Plus','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus1','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ = TwoPUtils.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=True)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [ ]:
h5list_all, frs_all = check_and_make_h5s(file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': h5list_all,
                               'save_path0': os.path.join(s2p_write_folder,mouse,"YMazeAll"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':frs_all[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'saveNWB':True,
                                'threshold_scaling':.9,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)

In [ ]:
_file_list = ({'date': '28_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '29_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':5},
             {'date': '01_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':7},
             {'date': '02_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':5},
             {'date': '05_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3},
             )

for f in _file_list:
    _h5list,_frs = check_and_make_h5s([f,])
    ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'nchannels':2,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2,
                                    'threshold_scaling':1.5,
                                    'chan2_thresh':.01})
    ops=s2p.run_s2p(ops=ops)
    !rm {_h5list[0]}/*.h5 

{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 39000
39000 39204
['/home/mplitt/2P_scratch/4467975.5/28_09_2020/YMaze_LNovel/YMaze_LNovel_001_004']
{}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['/home/mplitt/2P_scratch/4467975.5/28_09_2020/YMaze_LNovel/YMaze_LNovel_001_004/YMaze_LNovel_001_004.h5']
time 365.37 sec. Wrote 39204 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no

In [ ]:
_file_list = ({'date': '04_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '04_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':8},)

_h5list,_frs = check_and_make_h5s(_file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"04_10_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'threshold_scaling':1.5,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)
for h5 in _h5list:
    !rm {h5}/*.h5 

In [ ]:
_file_list = ({'date': '06_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':5},
             {'date': '06_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':9},)

_h5list,_frs = check_and_make_h5s(_file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"06_10_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'threshold_scaling':1.5,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)
for h5 in _h5list:
    !rm {h5}/*.h5 

In [ ]:
for h5 in h5list_all:
    !rm {h5}/*.h5 